# Demonstration of CG building of linear polymer chains

In [ ]:
# logging tools
import logging
logging.basicConfig(
    level=logging.WARNING,
    # level=logging.DEBUG,
    force=True,
)

from rich.progress import Progress, BarColumn, TimeRemainingColumn
def build_progress() -> Progress:
    return Progress(
        "[progress.description]{task.description}",
        BarColumn(),
        "(polymer {task.completed}/{task.total})",
        TimeRemainingColumn(),
        "Placing {task.fields[chain_len]}-mer chain",
        auto_refresh=False,
    )

# library imports
import numpy as np
from scipy.spatial.transform import Rotation, RigidTransform

from rdkit.Chem import Mol, MolFromSmiles, AddHs
from rdkit.Chem.rdMolTransforms import TransformConformer
from rdkit.Chem.AllChem import EmbedMolecule

from mupt.geometry.coordinates.reference import CoordAxis
from mupt.geometry.transforms.rigid import rigid_vector_coalignment

In [ ]:
from mupt.mupr.connection import ChainDirection


def mol_aligned_to_axis(smiles : str, axis : CoordAxis=CoordAxis.X) -> tuple[Mol, int, int]:
    '''
    Initialize a Mol with a pair of marked positions pinned to the x-axis, equidistant from the origin
    Returns the mol (with associated conformer), the bound conformer ID, and the semi-axis length
    '''
    mol = MolFromSmiles(smiles, sanitize=True)
    mol = AddHs(mol)
    conf_id : int = EmbedMolecule(mol) # positions are determined here
    conf = mol.GetConformer(conf_id)

    span_atom_pos : dict[str, np.ndarray] = {}
    for atom in mol.GetAtoms():
        if (map_num := atom.GetAtomMapNum()):
            span_atom_pos[map_num] = conf.GetAtomPosition(atom.GetIdx())
    pos1 = np.array(span_atom_pos[ChainDirection.ANTERO.value])
    pos2 = np.array(span_atom_pos[ChainDirection.RETRO.value])
    
    semimajor = np.linalg.norm(pos2 - pos1) / 2.0
    axis_vec = np.zeros(3, dtype=float)
    axis_vec[axis.value] = semimajor
    
    axis_alignment = rigid_vector_coalignment(
        vector1_start=pos1,
        vector1_end=pos2,
        vector2_start=np.zeros(3, dtype=float),
        vector2_end=axis_vec,
        t1=1/2,
        t2=0.0,
    )
    TransformConformer(conf, trans=axis_alignment.as_matrix())

    return mol, conf_id, semimajor

## Compile lexicon of unit repeat units (with fitted Ellipsoidal shapes)
Rigidly moves each set of atoms defining the repeat units to lie along the chosen axis; this is a "reference" orientation  

In this example, ths sizes of the beads are chosen to match the inter-atom distance defined by the atomic conformer;   
In the future, this relationship will be inverted, i.e. first setting the bead size, then pinning the pair of anchor atoms that distance apart while relaxing the interior of the conformer

In [ ]:
AXIS : CoordAxis = CoordAxis.X
ORIGIN : np.ndarray = np.array([0.0, 0.0, 0.0], dtype=float)
SEMIMINOR_FRACT : float = 0.5 # how long the pair of minor axes should be as a fraction of the major axis length

In [ ]:
from mupt.mupr.primitives import Primitive
from mupt.geometry.shapes import Ellipsoid
from mupt.interfaces.rdkit import primitive_from_rdkit


# TODO: further consolidate this to be read directly off of atom map numbers (once more easily customizable linkers are supported)
POS_1_IDX : int = ChainDirection.ANTERO.value
POS_2_IDX : int = ChainDirection.RETRO.value
rep_unit_smiles : dict[str, str] = {
    'head' : f'[OH:{POS_1_IDX}]c1ccc(cc1)S(=O)(=O)c1cc[c:{POS_2_IDX}](cc1)-*',
    'mid'  : f'*-[O:{POS_1_IDX}]c1ccc(cc1)S(=O)(=O)c1cc[c:{POS_2_IDX}](cc1)-*',
    'tail' : f'*-[O:{POS_1_IDX}]c1ccc(cc1)S(=O)(=O)c1ccc(cc1)[OH:{POS_2_IDX}]',
}
rep_unit_connection_tags : dict[str, tuple[ChainDirection, ...]] = {
    'head' : (ChainDirection.ANTERO,),
    'mid'  : (ChainDirection.RETRO, ChainDirection.ANTERO),
    'tail' : (ChainDirection.RETRO,),
}

lexicon : dict[str, Primitive] = {}
for unit_name, smiles in rep_unit_smiles.items():
    mol, conf_id, semimajor = mol_aligned_to_axis(smiles, axis=AXIS)
    semiminor = SEMIMINOR_FRACT * semimajor
    print(unit_name)
    display(mol)

    prim = primitive_from_rdkit(mol, conformer_idx=conf_id, label=unit_name)
    for chain_tag, connector in zip(rep_unit_connection_tags[unit_name], prim.connectors.values(), strict=True):
        connector.anchor = (chain_tag == ChainDirection.ANTERO) and ChainDirection.RETRO or ChainDirection.ANTERO # make opposite to that of linker
        connector.linkables.add(chain_tag)
        print(connector)

    radii = np.full(3, semiminor)
    radii[AXIS.value] = semimajor
    prim.shape = Ellipsoid(radii)

    lexicon[unit_name] = prim

## Construct hierarchy of Primitives representing polymer system
Hierarchy constructed here corresponds to abox of homopolymer chains with a given distribution of lengths;  
any given branch traced through the hierarchy is structured as Universe -> chains -> repeat units -> atoms

Each chain's units begin in reference position (i.e. level and centered at origin) initially, and are then placed by a placement generator  
An angle-constrained random walk is provided as an example implementation, but other implementations (e.g. DPD) can be slotted in just as easily by replacing the kind of builder user

In [ ]:
# configure system parameters
R_excl : float = 10.0
bond_length : float = 1.5 # 5.5
angle_max_rad : float = np.pi/4

n_chains : int = 4
dop_min : int = 5 # must be at least 2!
dop_max : int = 10

In [ ]:
import networkx as nx
from mupt.mupr.topology import TopologicalStructure

from mupt.builders.random_walk import AngleConstrainedRandomWalk
from mupt.geometry.coordinates.directions import random_unit_vector


univprim = Primitive(label='universe')
with build_progress() as progress:
    task = progress.add_task('Building chains', total=n_chains, chain_len=0)
    for chain_len in np.random.randint(dop_min, dop_max + 1, size=n_chains):
        progress.update(task, chain_len=chain_len)
        
        # build chain hierarchy
        molprim = Primitive(label=f'{chain_len}-mer_chain')
        unit_names : list[str] = ['head'] + ['mid']*(chain_len - 2) + ['tail']
        for i, unit_name in enumerate(unit_names)   :
            rep_unit_prim = lexicon[unit_name].copy()
            molprim.attach_child(rep_unit_prim)
            
        # assign path graph as topology, connecting adjacent Primitives
        molprim.set_topology(
            nx.path_graph(
                molprim.children_by_handle.keys(),
                create_using=TopologicalStructure,
            ),
            max_registration_iter=100,
        )

        # place beads by random walk (other build implementations would go here)
        direction = random_unit_vector()
        builder = AngleConstrainedRandomWalk(
            bond_length=bond_length,
            angle_max_rad=angle_max_rad,
            initial_point=R_excl*direction,
            initial_direction=direction,  # randomize starting direction to reduce likelihood of collisions
        )
        for handle, placement in builder.generate_placements(molprim):
            molprim.children_by_handle[handle].rigidly_transform(placement)
            
        # attach chain to universe, expand to show topology of beads
        mol_handle = univprim.attach_child(molprim)
        univprim.expand(mol_handle)
        
        progress.advance(task)
        progress.refresh()

univprim.visualize_topology()

## Flatten Primitive hierarchy and export as all-atom system
Expand each level until the hierarchy is just leaves below a root (atoms in a molecule system), then write out to RDKit

In [ ]:
from mupt.interfaces.rdkit import primitive_to_rdkit

chain : Mol = primitive_to_rdkit(univprim.copy()) # make copy for export, as flattening is done in-place
display(chain)

In [ ]:
from pathlib import Path
from rdkit.Chem import MolToMolFile


save_dir : Path = Path('mupt-built_systems')
save_dir.mkdir(exist_ok=True)
save_path : Path = save_dir / f'ellipsoidal_backmap_{n_chains}x[{dop_min}-{dop_max}]-mer_chains.mol'

MolToMolFile(chain, str(save_path))
print(f'Exported {chain.GetNumAtoms()}-atom system to {save_path}')

## Visualize placement of subprimitives [slow for larger systems, can skip if desired]
Visualize co-levels in hierarchy and plot residues (as Ellipsoids here) and individual atoms (as single points)

In [ ]:
# NBVAL_SKIP
from anytree.iterators import LevelOrderGroupIter

import matplotlib.pyplot as plt
# %matplotlib ipympl

scale : float=16
aspect : float = 1/2
plt.close()
fig = plt.figure(figsize=(scale, scale*aspect))
fig.canvas.header_visible = False

levels : dict[int, tuple[Primitive]] = {i : grp for i, grp in enumerate(LevelOrderGroupIter(univprim))}

ellip_depth : int = 1
ax_ellip = fig.add_subplot(1, 2, ellip_depth, projection='3d')
ax_ellip.autoscale(enable=True)
ax_ellip.set_title(f'Spatial placement of repeat units\n(as ellipsoids) (depth {ellip_depth})')
for res_prim in levels[ellip_depth]:
    _ = ax_ellip.plot_surface(*res_prim.shape.surface_mesh(10, 10).T, color='c', alpha=0.5)

atom_depth : int = 2
ax_atomic = fig.add_subplot(1, 2, atom_depth, projection='3d')
ax_atomic.autoscale(enable=True)
ax_atomic.set_title(f'Spatial placement of atom\n(as point particles) (depth {atom_depth})')
for res_prim in levels[atom_depth]:
    _ = ax_atomic.scatter(*res_prim.shape.positions.T, color='r', marker='.', alpha=0.5)

In [ ]:
# # optional; can apply a rigid transformation to the ENTIRE molecule at once - all sub-parts will be "carried along for the ride"
# random_rot = RigidTransform.from_rotation(Rotation.random())
# univprim.rigidly_transform(random_rot)